# SETUP

In [2]:
# установка внешних программ и библиотек на виртуальную машину
!git clone --recurse-submodules https://github.com/KarlsruheMIS/KaMIS.git
%cd KaMIS
!./compile_withcmake.sh

%cd ..


!pip3 install python-sat
#!sudo apt install minisat
!git clone https://github.com/arminbiere/kissat.git
%cd kissat
!./configure
!make

%cd ..

!cp KaMIS/deploy/redumis redumis
!cp KaMIS/deploy/online_mis online_mis
!cp kissat/build/kissat kissat1



Cloning into 'KaMIS'...
remote: Enumerating objects: 1492, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 1492 (delta 41), reused 51 (delta 29), pack-reused 1399
Receiving objects: 100% (1492/1492), 12.79 MiB | 2.96 MiB/s, done.
Resolving deltas: 100% (832/832), done.
/root/science/BorsukResearch/BoolCubeN10Diam6/clean_code/KaMIS
./compile_withcmake.sh: line 17: cmake: command not found
make: *** No targets specified and no makefile found.  Stop.
cp: cannot stat './build/redumis': No such file or directory
cp: cannot stat './build/graphchecker': No such file or directory
cp: cannot stat './build/sort_adjacencies': No such file or directory
cp: cannot stat './build/online_mis': No such file or directory
cp: cannot stat './build/wmis/branch_reduce': No such file or directory
cp: cannot stat './build/wmis/weighted_ls': No such file or directory
/root/science/BorsukResearch/BoolCubeN10Diam6/clean_code

[notice] A new relea

In [3]:
# функции из библиотеки
# раскраска графа в формате  DIMACS, преобразование в CNF

import math
import numpy as np
from numpy import linalg as LA
import networkx as nx

#import matplotlib.pylab as plt

#from google.colab import files 
#files.upload() 
#files.download("file.txt")

#https://pysathq.github.io/
#!pip install python-sat
#import pysat
from pysat.formula import CNF
from pysat.solvers import *

import os
import sys
import random
from zipfile import ZipFile
#from tqdm import tqdm_notebook as tqdm
from itertools import combinations, permutations
import glob

out_dir = ""

class Utils:
    
    def read_dimacs_graph(file = 'graph.col'):
        
        if not (os.path.exists(file) and os.path.getsize(file) > 0):        
            raise Exception("File " + file + " not found")
        
        nodes = []    
        edges = []
        labels = []
        got_labels = False
        nnodes = nedges = 0
        
        with open(file, 'r') as f:
            for line in f:
                line = [l.strip() for l in line.split(' ')]
                if line[0] == 'c': #comment
                    continue
                elif line[0] == 'p': #problem
                    nnodes = int(line[2])
                    nedges = int(line[3])
                    nodes = list(range(1, nnodes + 1))
                    labels = [0] * nnodes
                elif line[0] == 'e': #edge
                    edges.append((int(line[1]), int(line[2])))
                elif line[0] == 'l':
                    labels[int(line[1]) - 1] = int(line[2])
                    got_labels = True

        if got_labels:        
            nodes = [(n, {'c' : l}) for n, l in zip(nodes, labels)]

        g = nx.Graph()
        g.add_nodes_from(nodes)
        g.add_edges_from(edges)
        return g

    def write_dimacs_graph(file = 'graph.col', g = nx.Graph(), comments = []):
        with open(file, 'w') as f:
            for c in comments:
                f.write("c " + c + "\n")
            f.write("p EDGE {} {}\n".format(g.number_of_nodes(), g.number_of_edges()))
            for u, v in g.edges():
                f.write("e {} {}\n".format(u, v))
            for node in g.nodes():
                if 'c' in g.node[node]:
                    f.write("l {} {}\n".format(node, g.node[node]['c']))

    def draw_with_colors(g = nx.Graph()):
        color_map = []
        for node in g.nodes():
            if 'c' in g.node[node]:
                color_map.append(g.node[node]['c'] * 10)            
        nx.draw(g, pos = nx.spring_layout(g, scale=2), node_color=color_map, with_labels=True, cmap = plt.cm.jet)

    def write_proof(file = "proof.txt", proof = []):
        with open(file, 'w') as f:
            for p in proof:
                f.write("%s\n" % str(p))

    def zip_files(file = "archive.zip", files = []):
        with ZipFile(file, 'w') as archive:
            for f in set(files):                
                if not (os.path.exists(file) and os.path.getsize(file) > 0):        
                    raise Exception("File " + file + " not found")
                archive.write(f)
                
def find_triangle(g = nx.Graph()):
    for a in g:
        for b, c in combinations(g[a], 2):
            if b in g[c]:
                return [a, b, c]
    return []
             
    #return set(frozenset([a, b, c]) for a in g for b, c in combinations(g[a], 2) if b in g[c])


def find_isolates(g = nx.Graph()):
    isolates = []
    for n in g:
        if g.degree(n) == 0:
            isolates.append(n)
    return isolates

class ColMap:
    
    def __init__(self, g = nx.Graph(), ncolors = 40):
        
        self.ncolors = ncolors
        self.cmap = dict()
        self.cunmap = dict()
    
        i = 1
        for node in g.nodes():
            for color in range(1, ncolors + 1):            
                self.cmap[(node, color)] = i
                self.cunmap[i] = (node, color)
                i += 1    

    def enc(self, node, color):
        return self.cmap[(node, color)]

    def dec(self, node_color):
        return self.cunmap[node_color]

class ColSAT:
   
    def __init__(self, g = nx.Graph(), ncolors = 10):
        
        self.ncolors = ncolors
        self.g = g.copy()
        self.cmap = ColMap(g, ncolors)        

    def check_coloring(self):
        for n1, n2 in self.g.edges():
            if 'c' not in self.g.node[n1] or 'c' not in self.g.node[n2]:
                return False
            if self.g.node[n1]['c'] == self.g.node[n2]['c']:
                return False
        return True
    
    def apply_model(self):
        
        check = set()
        for var in self.model[self.model > 0]:        
            node, color = self.cmap.dec(var)
            self.g.node[node]['c'] = color
            if (node, color) in check:
                raise Exception("Two colors for one node???")
            else:
                check.add((node, color))
        
        self.colored = self.check_coloring()
        
        if self.colored != self.solved:
            raise Exception("Something went wrong!")
        
        return self.colored
        
    def build_cnf(self, ):
        
        self.formula = CNF()
        colors = list(range(1, self.ncolors + 1))    

        maxc = -1        # condition k<12 is not for general case
        k = 0
        for clique in nx.find_cliques(self.g):
            if len(clique)>maxc:
                maxclique = clique.copy()  
                maxc = len(clique)
            if k<12:
                k += 1
                continue
           
            col = 1
            if len(maxclique)>self.ncolors:
                print("Size of the maximal clique is {}".format(len(maxclique)))
                raise Exception("Size of the maximal clique is {} > {}".format(len(maxclique),ncolors))
            for v in maxclique:
                self.formula.append([self.cmap.enc(v, col)])
                col += 1
            break

        for n1, n2 in self.g.edges():
            for c in colors:            
                self.formula.append([-self.cmap.enc(n1, c), -self.cmap.enc(n2, c)])


        for n in self.g.nodes():
            #if not n in specials:
            self.formula.append([self.cmap.enc(n, c) for c in colors])
            #for c1 in colors:
            #    for c2 in colors:
            #        if c1 < c2:
            #            self.formula.append([-self.cmap.enc(n, c1), -self.cmap.enc(n, c2)])
        
        return self.formula
    
    def solve_cnf(self, solver = ''):
        
        triangle = find_triangle(self.g)
        assumptions = []
        if len(triangle) > 0:            
            assumptions = [self.cmap.enc(triangle[0], 1), self.cmap.enc(triangle[1], 2), self.cmap.enc(triangle[2], 3)]
            
        #Glucose3, Glucose4, Lingeling, MapleChrono, MapleCM, Maplesat, Minisat22, MinisatGH
        #with Glucose4(bootstrap_with=self.formula.clauses, with_proof=True) as ms:        
        with Lingeling(bootstrap_with=self.formula.clauses) as ms:
            self.solved = ms.solve(assumptions=assumptions)
            if self.solved:
                self.model = np.array(ms.get_model())
                self.apply_model()
            else:                
                self.proof = []#ms.get_proof()
                self.colored = False
                
        return self.solved

In [4]:
# выбор графа на вершинах гиперкуба (покрышки), удаление независимого множества,
# проверка существования раскраски

import networkx as nx

def dist(a,b):
  return sum([int(c) for c in bin(a^b).split('b')[1]])

def bin_graph(n, k, trim = []):   # третий параметр - множество вершин
  g = nx.Graph()                  # по k-окрестностям которых происходит отсечение    
  for i in range(0,2**n):
    if dist(0,i)%2==1:
      continue
    flag = True
    for t in trim:
      if dist(t,i)>k:
        flag = False
        break
    if flag:
      g.add_node(i)
  for i in list(g.nodes()):
    for j in g.nodes():
      if (i<j) and (dist(i,j)==k):
        g.add_edge(i,j)
  return nx.convert_node_labels_to_integers(g)

def small_graph(n, k, dists = [2,4,8]):   # третий параметр - множество вершин
  g = nx.Graph()                  # по k-окрестностям которых происходит отсечение    
  for i in range(0,2**n):
    if dist(0,i) not in dists:
      continue
    g.add_node(i)
  for i in list(g.nodes()):
    for j in g.nodes():
      if (i<j) and (dist(i,j)==k):
        g.add_edge(i,j)
  return nx.convert_node_labels_to_integers(g)

def bin_graph1(n, k, trim = []):
  g = nx.Graph()
  for i in range(0,2**n):
    if dist(0,i)%2==1:
      continue
    flag = True
    if dist(0,i)>k:
      flag = False
      
    if flag or i in set(trim):
      g.add_node(i)
  for i in list(g.nodes()):
    for j in g.nodes():
      if (i<j) and (dist(i,j)==k):
        g.add_edge(i,j)
  return nx.convert_node_labels_to_integers(g)


def r_graph(n, k):      # граф запретов,
  g = nx.Graph()        # ребрами соединены вершины на расстоянии >k
  for i in range(0,2**n):
    g.add_node(i)
  for i in list(g.nodes()):
    for j in g.nodes():
      if (i<j) and (dist(i,j)>k):
        g.add_edge(i,j)
  return g

def write_metis(g0,fname):    # запись графа во входном формате KaMIS
  g = nx.convert_node_labels_to_integers(g0)
  with open(fname,"w") as f:
    f.write("{} {}\n".format(g.number_of_nodes(),g.number_of_edges()))
    for v in range(g.number_of_nodes()):
      f.write(" ".join([str(i+1) for i in sorted(list(g[v]))])+"\n")    

def write_cnf(g,ncolors,filename):  # запись КНФ для SAT-решателя    
  problem = ColSAT(g, ncolors)
  problem.build_cnf().to_file(filename)

def select_mis(g,filename):  # возвращает независимое множество
  g1 = g.copy()
  with open(filename,"r") as f:
    l = f.readlines()
    for i in range(len(l)):
      if int(l[i])==0:
        g1.remove_node(i)    
  return nx.convert_node_labels_to_integers(g1,0)      


def delete_mis(g,filename):  # удаляет независимосе множество
  g1 = g.copy()
  with open(filename,"r") as f:
    l = f.readlines()
    for i in range(len(l)):
      if int(l[i])==1:
        g1.remove_node(i)    
  return nx.convert_node_labels_to_integers(g1,0)      

In [5]:
FILENAME = "test_11"
COLORSS = 0

def is_not_colorable(graph, color_count, timer):
    global FILENAME, COLORSS
    COLORSS += 1
    coloring_filename = FILENAME + ".cnf"
    try:
        graph_copy = graph.copy()
        write_cnf(graph_copy, color_count, coloring_filename)
    except Exception as err:
        return True

    outputt = !timeout $timer ./kissat1 $coloring_filename
    if any(["UNSATISFIABLE" in line for line in outputt]):
         return True
    return False

def is_colorable(graph, color_count, timer):
    global FILENAME, COLORSS
    COLORSS += 1
    coloring_filename = FILENAME + ".cnf"
    try:
        graph_copy = graph.copy()
        write_cnf(graph_copy, color_count, coloring_filename)
    except Exception as err:
        return False

    outputt = !timeout $timer ./kissat1 $coloring_filename
    if all(["UNSATISFIABLE" not in line for line in outputt]):
        if any(["SATISFIABLE" in line for line in outputt]):
            return True
    return False

def check_colorable_fast(graph, max_colors=100, timer=120):
    if graph.number_of_edges() == 0:
        return (None, 0)

    max_uncolorable = None
    min_colorable = None
    
    left_border = 1
    right_border = max_colors + 1
    while right_border - left_border > 1:
        color_count_to_check = (right_border + left_border) // 2
        if is_not_colorable(graph, color_count_to_check, timer):
            left_border = color_count_to_check
        else:
            right_border = color_count_to_check
    max_uncolorable = left_border

    left_border = 1
    right_border = max_colors
    while right_border - left_border > 1:
        color_count_to_check = (right_border + left_border) // 2
        if is_colorable(graph, color_count_to_check, timer):
            right_border = color_count_to_check
        else:
            left_border = color_count_to_check
    if right_border == max_colors:
        if is_colorable(graph, max_colors, timer):
            min_colorable = right_border 
        return (max_uncolorable, min_colorable)
    else:
        min_colorable = right_border
        return (max_uncolorable, min_colorable)
    return (None, None)

# Some theory

k6 = set of v on dist of 6 from 0. k6 is sorted by decreasing number of edges to the fixed starting set(usually named kv or base_conf)

# SOLVE NO 3-CLIQUE ( need to check)

tmp text commented in this slide
<!-- \begin{prop}
Граф без треугольников красится в 11 цветов
\end{prop}
Что для этого делается? Не умаляя общности есть ребро. Один его конец - это 0, другой его конец это 1 = 111 111 000 0.
Разобьем все множество вершин на следующие подмножества:
вершины на расстоянии 2 от 0. = B
вершины на расстоянии 4 от 0, которые лежат на расстоянии 6 от 1 = С
вершины на расстоянии 4 от 0, которые не лежат на расстоянии 6 от 1 = D
Вершины на расстоянии 6 от 0(они не соседи с "1" так как иначе есть треугольник) = E
0 + B - 9 цвет
1 + E - 8 цвет
С - 7 цвет(нет ребер, так как иначе треугольник с "1")
Все множество вершин D красится в 6 цветов -->


In [6]:
first = 0b1111110000
vertexes = []
for i in range(1, 1024):
    if dist(i, 0) % 2 == 1:
        continue
    if dist(i, 0) == 4 and dist(first, i) < 6:
        vertexes.append(i)

g = nx.Graph()


for vertex in vertexes:
    g.add_node(vertex)
for i in list(g.nodes()):
    for j in g.nodes():
        if (i<j) and (dist(i,j)==6):
            g.add_edge(i,j)

print(is_colorable(nx.convert_node_labels_to_integers(g), 6, 20))

True


# SOLVE NO 4-CLIQUE

In [ ]:
BIG_NUM = 3
COUNTER = 0
SAVED = []
BPM = [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]

# Not simple, tbh. This is somewhat next-level version of "cook graph" function
def simple_cook_graph_no_K4(base_conf, add_conf, kl3s, k_2, k_4, k_6, n=10, k=6):   # третий параметр - множество вершин
    global COUNTER, BPM
    COUNTER += 1
    if COUNTER % 50 == 0:
        print(BPM)
    g = nx.Graph() 
    bans = base_conf + add_conf
    pooles = k_2 + k_4 + k_6
    for i in bans:
        g.add_node(i)
    
    good_vrtx = []
    for i in pooles:
        flag = True
        for j in bans:
            if dist(i,j) > k or dist(i, j) == 0:
                flag = False
                break
        if not flag:
            continue

        big_flag = True
        for kl3 in kl3s:
            fflag = False
            for v in kl3:
                if dist(i, v) < 6:
                    fflag = True
                    break
            if not fflag:
                big_flag = False
                break
        if big_flag:
            g.add_node(i)
            good_vrtx.append(i)

    prepared = []
    for i in bans:
        for j in bans:
            if (i<j) and (dist(i,j)==k):
                g.add_edge(i,j)
                prepared.append((i, j))
    
    for i in bans:
        for j in good_vrtx:
            if (i<j) and (dist(i,j)==k):
                g.add_edge(i,j)

    for i in good_vrtx:
        for j in bans:
            if (i<j) and (dist(i,j)==k):
                g.add_edge(i,j)

    for i in good_vrtx:
        for j in good_vrtx:
            if (i<j) and (dist(i,j)==k):
                flag = True
                for prep in prepared:
                    if dist(i, prep[0]) == k and dist(i, prep[1]) == k and dist(j, prep[0]) == k and dist(j, prep[1]) == k:
                        flag = False
                if flag:
                    g.add_edge(i,j)
    

    # for i in list(g.nodes()):
    #     for j in g.nodes():
    #         if (i<j) and (dist(i,j)==k):
    #             g.add_edge(i,j)
    return nx.convert_node_labels_to_integers(g)

def sort_to_check(to_check, base_conf, add_conf):
    to_to = [[a, 0] for a in to_check]
    for a in to_to:
        for b in base_conf:
            if dist(a[0], b) == 6:
                a[1] += 1
        for b in  add_conf:
            if dist(a[0], b) == 6:
                a[1] += 1
    tbd = sorted(to_to, key=lambda x: (-x[1], x[0]))
    return [a[0] for a in tbd]


def bruteforce_nolimits_nok4(to_check, base_conf, add_conf, k_2, k_4, k2s = [], kl3s=[], how_much=BIG_NUM, timer=120, n=10, k=6, fromm=0):
    global COUNTER, BPM, SAVED
    to_check = sort_to_check(to_check, base_conf, add_conf)
    if how_much == BIG_NUM:
        to_check = to_check[fromm:]
    ln = len(to_check)
    for i in range(ln):
        BPM[how_much][0] = i
        BPM[how_much][1] = ln
        # if i % 20 == 0 and how_much == BIG_NUM - 1:
        #     print(i, len(to_check))
        elem = to_check[i]
        if elem in base_conf or elem in add_conf:
            continue
        flag = True
        for el in add_conf:
            if dist(elem, el) > 6:
                flag = False
                break
        if not flag:
            continue
        big_flag = True
        for kl3 in kl3s:
            fflag = False
            for v in kl3:
                if dist(elem, v) < 6:
                    fflag = True
                    break
            if not fflag:
                big_flag = False
                break
        if not big_flag:
            continue
    
        kl3s_updated = []
        for k2ss in k2s:
            if dist(elem, k2ss[0]) == 6 and dist(elem, k2ss[1]) == 6:
                kl3s_updated.append((k2ss[0], k2ss[1], elem))
        k2s_updated = []
        for v in base_conf + add_conf:
            if dist(elem, v) == 6:
                k2s_updated.append((elem, v))
        if how_much > 1:
            bruteforce_nolimits_nok4(to_check[i+1:], base_conf, add_conf + [elem], k_2=k_2,
                                     k_4=k_4, k2s = k2s + k2s_updated, kl3s = kl3s + kl3s_updated,
                                     how_much=how_much-1,timer=timer, n=n, k=k)
        else:
            grph = simple_cook_graph_no_K4(base_conf=base_conf, add_conf=add_conf + [elem], kl3s = kl3s + kl3s_updated, k_2=k_2, k_4=k_4, k_6=to_check[i+1:], n=n, k=k)
            brb = is_colorable(grph, 11, timer)
            if not brb:
                SAVED.append((base_conf, add_conf + [elem]))
                print(base_conf, add_conf + [elem])
    if how_much == BIG_NUM:
        print("analysing {} finished".format(BIG_NUM))

def bruteforce_nolimits_nok4_short(to_check, base_conf, add_conf, k_2, k_4, k2s = [], kl3s=[], how_much=4, timer=120, n=10, k=6, fromm=0):
    global COUNTER, BPM, SAVED
    to_check = sort_to_check(to_check, base_conf, add_conf)
    if how_much == BIG_NUM:
        to_check = to_check[fromm:]
    ln = len(to_check)
    for i in range(ln):
        BPM[how_much][0] = i
        BPM[how_much][1] = ln
        elem = to_check[i]
        if elem in base_conf or elem in add_conf:
            continue
        flag = True
        for el in add_conf:
            if dist(elem, el) > 6:
                flag = False
                break
        if not flag:
            continue
        big_flag = True
        for kl3 in kl3s:
            fflag = False
            for v in kl3:
                if dist(elem, v) < 6:
                    fflag = True
                    break
            if not fflag:
                big_flag = False
                break
        if not big_flag:
            continue
    
        kl3s_updated = []
        for k2ss in k2s:
            if dist(elem, k2ss[0]) == 6 and dist(elem, k2ss[1]) == 6:
                kl3s_updated.append((k2ss[0], k2ss[1], elem))
        k2s_updated = []
        for v in base_conf + add_conf:
            if dist(elem, v) == 6:
                k2s_updated.append((elem, v))

        if how_much == 1:
            grph = simple_cook_graph_no_K4(base_conf=base_conf, add_conf=add_conf + [elem], kl3s = kl3s + kl3s_updated, k_2=k_2, k_4=k_4, k_6=[], n=n, k=k)
            brb = is_colorable(grph, 11, timer)
            if not brb:
                print(base_conf, add_conf + [elem])
                SAVED.append((base_conf, add_conf + [elem]))
        if how_much > 1:
            bruteforce_nolimits_nok4_short(to_check[i+1:], base_conf, add_conf + [elem], k_2=k_2,
                                     k_4=k_4, k2s = k2s + k2s_updated, kl3s = kl3s + kl3s_updated,
                                     how_much=how_much-1,timer=timer, n=n, k=k)

    if how_much == BIG_NUM:
        print("analysing {}-or-less-but-at-least-1 finished".format(BIG_NUM))

In [ ]:
FILENAME = "NoK4"
# !!! this is late-stage thing
base_conf = [0, 945, 910]

k2s = [(0, 945),
       (0, 910),
       (945, 910)]
kl3s = [(0, 910, 945)]

k_2 = []
k_4 = []
k_6 = []

for i in range(2**10):
    if dist(0, i) % 2 == 1:
        continue
    flag = True
    for base_v in base_conf:
        if dist(i, base_v) == 0 or dist(i, base_v) > 6:
            flag = False
            continue
    big_flag = True
    for kl3 in kl3s:
        fflag = False
        for v in kl3:
            if dist(i, v) < 6:
                fflag = True
                break
        if not fflag:
            big_flag = False
            break
    if not big_flag:
        continue
    if flag:
        if dist(0, i) == 2:
            k_2.append(i)
        if dist(0, i) == 4:
            k_4.append(i)
        if dist(0, i) == 6:
            k_6.append(i)

print(len(k_6), len(k_4), len(k_2))

+4+ [:80]

In [ ]:
%%time
BIG_NUM=4
SAVED = []
print("COLORED", COLORSS)
print('to check:', len(k_6), len(k_6))
bruteforce_nolimits_nok4(k_6[:80], kv, [], k_2+k_6[80:], k_4, k2s, kl3s, BIG_NUM, 40, 10, 6)
print("AFTER ALL-COLORED", COLORSS)

+ exactly 1 or 2 in [:]

In [ ]:
%%time
BIG_NUM=2
print("COLORED", COLORSS)
print('to check:', len(k_6), len(k_6))
bruteforce_nolimits_nok4_short(k_6, kv, [], k_2, k_4, k2s, kl3s, BIG_NUM, 20, 10, 6)
print("AFTER ALL-COLORED", COLORSS)

In [ ]:
%%time
BIG_NUM=1
print("COLORED", COLORSS)
print('to check:', len(k_6), len(k_6))
bruteforce_nolimits_nok4_short(k_6, kv, [], k_2, k_4, k2s, kl3s, BIG_NUM, 20, 10, 6)
print("AFTER ALL-COLORED", COLORSS)

And we are left with exactly 0,1,2,3 in [:80]. + cases when it is not colored in such conditions.

In [ ]:
%%time
BIG_NUM=3
SAVED = []
print("COLORED", COLORSS)
print('to check:', len(k_6), len(k_6))
bruteforce_nolimits_nok4_short(k_6[:80], kv, [], k_2+k_6[80:], k_4, k2s, kl3s, BIG_NUM, 40, 10, 6)
print("AFTER ALL-COLORED", COLORSS)

In [ ]:
%%time
BIG_NUM=2
print("COLORED", COLORSS)
print('to check:', len(k_6), len(k_6))
bruteforce_nolimits_nok4_short(k_6[:80], kv, [], k_2+k_6[80:], k_4, k2s, kl3s, BIG_NUM, 40, 10, 6)
print("AFTER ALL-COLORED", COLORSS)

In [ ]:
%%time
BIG_NUM=1
print("COLORED", COLORSS)
print('to check:', len(k_6), len(k_6))
bruteforce_nolimits_nok4_short(k_6[:80], kv, [], k_2+k_6[80:], k_4, k2s, kl3s, BIG_NUM, 40, 10, 6)
print("AFTER ALL-COLORED", COLORSS)

In [ ]:
def build_blocks_by_conf(conf):
    vrtx = conf[0] + conf[1]
    lss = len(vrtx)
    k22 = []
    k33 = []
    for i in range(lss):
        for j in range(i+1, lss):
            if dist(vrtx[i], vrtx[j]) == 6:
                k22.append((vrtx[i], vrtx[j]))
    for i in range(lss):
        for j in range(i+1, lss):
            for j1 in range(j+1, lss):
                if dist(vrtx[i], vrtx[j]) == 6 and dist(vrtx[i], vrtx[j1]) == 6 and dist(vrtx[j1], vrtx[j]) == 6:
                    k33.append((vrtx[i], vrtx[j], vrtx[j1]))
    return k22, k33

+1/+2 exactly in [:] is already solved. So only "at least 3 in [:] left, even when 1,2,3 in [:80] is taken already."
This task could be splited on 2: exactly 3 in [:] AND at least 4 in [:]

In [ ]:
%%time

#exactly +3 in [:]
SSAVED = []
for gr in SAVED:
    if len(gr[1]) == 3:
        k2_ss, k3_ss = build_blocks_by_conf(gr)
        grph = simple_cook_graph_no_K4(base_conf=gr[0], add_conf=gr[1], kl3s = k3_ss, k_2=k_2, k_4=k_4, k_6=[], n=10, k=6)
        brb = is_colorable(grph, 11, 20)
        if not brb:
            print(gr)
            SSAVED.append((gr[0], gr[1]))
    else:
        k2_ss, k3_ss = build_blocks_by_conf(gr)
        BIG_NUM = 3 - len(gr[1])
        bruteforce_nolimits_nok4_short(to_check=k_6[80:], base_conf=gr[0], add_conf=gr[1], k_2=k_2, k_4=k_4, k2s=k2_ss, kl3s=k2_ss, how_much=BIG_NUM, timer=20, n=10, k=6)

In [ ]:
print(len(SSAVED), len(SAVED))

In [ ]:
%%time

#решаем вопрос о ровно 3 вершинах для случая (1 из первых 80), (+2 из остальных). Этот случай вложен в (1 из первых 80) (+1+ из остальных)
SSAVED = []
for gr in SAVED:
    if len(gr[1]) == 1:
        k2_ss, k3_ss = build_blocks_by_conf(gr)
        BIG_NUM = 2 - len(gr[1])
        bruteforce_nolimits_nok4(to_check=k_6[80:], base_conf=gr[0], add_conf=gr[1], k_2=k_2, k_4=k_4, k2s=k2_ss, kl3s=k2_ss, how_much=BIG_NUM, timer=20, n=10, k=6)
 

In [ ]:
print(len(SSAVED), len(SAVED))

In [ ]:
# Solving question about exactly 0 in [:80], at least 3 at [:]. This case is inside of "at least 1 in remaining [80:]"

#решаем вопрос о ровно 3 вершинах для случая (>>0<< из первых 80)
## вместо этого добираем +1+ из оставшихся на расстоянии 6
k2_ss, k3_ss = build_blocks_by_conf([[0, 945, 910],[]])
BIG_NUM = 1
bruteforce_nolimits_nok4(to_check=k_6[80:], base_conf=[0, 945, 910], add_conf=[], k_2=k_2, k_4=k_4, k2s=k2_ss, kl3s=k2_ss, how_much=BIG_NUM, timer=20, n=10, k=6)

In [ ]:
%%time

#решаем вопрос о хотя бы 5 вершинах для случаев (>>0,1 разобраны выше на самом деле<<,2,3 из первых 80)
## вместо этого добираем +1+ из оставшихся на расстоянии 6
for gr in SAVED:
    if len(gr[1]) == 2 or  len(gr[1]) == 3:
        k2_ss, k3_ss = build_blocks_by_conf(gr)
        BIG_NUM = 1
        bruteforce_nolimits_nok4(to_check=k_6[80:], base_conf=gr[0], add_conf=gr[1], k_2=k_2, k_4=k_4, k2s=k2_ss, kl3s=k2_ss, how_much=BIG_NUM, timer=20, n=10, k=6)
 

In [ ]:
print(len(SSAVED), len(SAVED))

# SOLVE NO 4-CLIQUE-1, but 4-CLIQUE-2

In [ ]:
base_conf = [0, 945, 910, 630]

k2s = [(0, 945),
       (630, 910),
       (0, 910),
       (0, 630),
       (630, 945),
       (945, 910)]
kl3s = [(0, 630, 910),
        (0, 630, 945),
        (630, 910, 945),
        (0, 910, 945)]
kl4s = [(0, 945, 910, 630)]

k_2 = []
k_4 = []
k_6 = []

for i in range(2**10):
    if dist(0, i) % 2 == 1:
        continue
    flag = True
    for base_v in base_conf:
        if dist(i, base_v) == 0 or dist(i, base_v) > 6:
            flag = False
            continue
    big_flag = True
    for kl3 in kl3s:
        res = 0
        for v in kl3:
            res ^= v
        if res == i:
            big_flag = False
            break
    if not big_flag:
        continue
    for kl4 in kl4s:
        fflag = False
        for v in kl4:
            if dist(i, v) < 6:
                fflag = True
                break
        if not fflag:
            big_flag = False
            break
    if not big_flag:
        continue
    if flag:
        if dist(0, i) == 2:
            k_2.append(i)
        if dist(0, i) == 4:
            k_4.append(i)
        if dist(0, i) == 6:
            k_6.append(i)

In [ ]:
BIG_NUM = 3
COUNTER = 0
BPM = [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]

def simple_cook_graph_no_K4_second(base_conf, add_conf, kl3s, kl4s, k_2, k_4, k_6, n=10, k=6):   # третий параметр - множество вершин
    global COUNTER, BPM
    COUNTER += 1
    if COUNTER % 50 == 0:
        print(BPM)
    g = nx.Graph() 
    bans = base_conf + add_conf
    pooles = k_2 + k_4 + k_6
    for i in bans:
        g.add_node(i)
    
    for i in pooles:
        flag = True
        for j in bans:
            if dist(i,j) > k or dist(i, j) == 0:
                flag = False
                break
        if not flag:
            continue

        big_flag = True
        for kl3 in kl3s:
            res = 0
            for v in kl3:
                res ^= v
            if res == i:
                big_flag = False
                break
        for kl4 in kl4s:
            fflag = False
            for v in kl4:
                if dist(i, v) < 6:
                    fflag = True
                    break
            if not fflag:
                big_flag = False
                break
        if big_flag:
            g.add_node(i)
    for i in list(g.nodes()):
        for j in g.nodes():
            if (i<j) and (dist(i,j)==k):
                g.add_edge(i,j)
    return nx.convert_node_labels_to_integers(g)

def sort_to_check(to_check, base_conf, add_conf):
    to_to = [[a, 0] for a in to_check]
    for a in to_to:
        for b in base_conf:
            if dist(a[0], b) == 6:
                a[1] += 1
        for b in  add_conf:
            if dist(a[0], b) == 6:
                a[1] += 1
    tbd = sorted(to_to, key=lambda x: (-x[1], x[0]))
    return [a[0] for a in tbd]


def bruteforce_nolimits_nok4_second(to_check, base_conf, add_conf, k_2, k_4, k2s = [], kl3s=[], kl4s=[], how_much=BIG_NUM, timer=120, n=10, k=6, fromm=0):
    global COUNTER, BPM
    to_check = sort_to_check(to_check, base_conf, add_conf)
    if how_much == BIG_NUM:
        to_check = to_check[fromm:]
    ln = len(to_check)
    for i in range(ln):
        BPM[how_much][0] = i
        BPM[how_much][1] = ln
        # if i % 20 == 0 and how_much == BIG_NUM - 1:
        #     print(i, len(to_check))
        elem = to_check[i]
        if elem in base_conf or elem in add_conf:
            continue
        flag = True
        for el in add_conf:
            if dist(elem, el) > 6:
                flag = False
                break
        if not flag:
            continue
        big_flag = True
        for kl3 in kl3s:
            res = 0
            for v in kl3:
                res ^= v
            if res == elem:
                big_flag = False
                break
        if not big_flag:
            continue
        for kl4 in kl4s:
            fflag = False
            for v in kl4:
                if dist(elem, v) < 6:
                    fflag = True
                    break
            if not fflag:
                big_flag = False
                break
        if not big_flag:
            continue
    
        kl4s_updated = []
        for k3ss in kl3s:
            if dist(elem, k3ss[0]) == 6 and dist(elem, k3ss[1]) == 6 and dist(elem, k3ss[2]):
                kl4s_updated.append((k3ss[2], k3ss[0], k3ss[1], elem))
        kl3s_updated = []
        for k2ss in k2s:
            if dist(elem, k2ss[0]) == 6 and dist(elem, k2ss[1]) == 6:
                kl3s_updated.append((k2ss[0], k2ss[1], elem))
        k2s_updated = []
        for v in base_conf + add_conf:
            if dist(elem, v) == 6:
                k2s_updated.append((elem, v))
        if how_much > 1:
            bruteforce_nolimits_nok4_second(to_check[i+1:], base_conf, add_conf + [elem], k_2=k_2,
                                     k_4=k_4, k2s = k2s + k2s_updated, kl3s = kl3s + kl3s_updated, kl4s = kl4s + kl4s_updated,
                                     how_much=how_much-1,timer=timer, n=n, k=k)
        else:
            grph = simple_cook_graph_no_K4_second(base_conf=base_conf, add_conf=add_conf + [elem], kl3s = kl3s + kl3s_updated, kl4s = kl4s + kl4s_updated, k_2=k_2, k_4=k_4, k_6=to_check[i+1:], n=n, k=k)
            brb = is_colorable(grph, 11, timer)
            if not brb:
                print(base_conf, add_conf + [elem])
    if how_much == BIG_NUM:
        print("analysing {} finished".format(BIG_NUM))

def bruteforce_nolimits_nok4_second_2(to_check, base_conf, add_conf, k_2, k_4, k2s = [], kl3s=[], kl4s=[], how_much=BIG_NUM, timer=120, n=10, k=6, fromm=0):
    global COUNTER, BPM
    to_check = sort_to_check(to_check, base_conf, add_conf)
    if how_much == BIG_NUM:
        to_check = to_check[fromm:]
    ln = len(to_check)
    if how_much == BIG_NUM:
        ln = ln // 2
    for i in range(ln):
        BPM[how_much][0] = i
        BPM[how_much][1] = ln
        # if i % 20 == 0 and how_much == BIG_NUM - 1:
        #     print(i, len(to_check))
        elem = to_check[i]
        if elem in base_conf or elem in add_conf:
            continue
        flag = True
        for el in add_conf:
            if dist(elem, el) > 6:
                flag = False
                break
        if not flag:
            continue
        big_flag = True
        for kl3 in kl3s:
            res = 0
            for v in kl3:
                res ^= v
            if res == elem:
                big_flag = False
                break
        if not big_flag:
            continue
        for kl4 in kl4s:
            fflag = False
            for v in kl4:
                if dist(elem, v) < 6:
                    fflag = True
                    break
            if not fflag:
                big_flag = False
                break
        if not big_flag:
            continue
    
        kl4s_updated = []
        for k3ss in kl3s:
            if dist(elem, k3ss[0]) == 6 and dist(elem, k3ss[1]) == 6 and dist(elem, k3ss[2]):
                kl4s_updated.append((k3ss[2], k3ss[0], k3ss[1], elem))
        kl3s_updated = []
        for k2ss in k2s:
            if dist(elem, k2ss[0]) == 6 and dist(elem, k2ss[1]) == 6:
                kl3s_updated.append((k2ss[0], k2ss[1], elem))
        k2s_updated = []
        for v in base_conf + add_conf:
            if dist(elem, v) == 6:
                k2s_updated.append((elem, v))
        if how_much > 1:
            bruteforce_nolimits_nok4_second(to_check[i+1:], base_conf, add_conf + [elem], k_2=k_2,
                                     k_4=k_4, k2s = k2s + k2s_updated, kl3s = kl3s + kl3s_updated, kl4s = kl4s + kl4s_updated,
                                     how_much=how_much-1,timer=timer, n=n, k=k)
        else:
            grph = simple_cook_graph_no_K4_second(base_conf=base_conf, add_conf=add_conf + [elem], kl3s = kl3s + kl3s_updated, kl4s = kl4s + kl4s_updated, k_2=k_2, k_4=k_4, k_6=to_check[i+1:], n=n, k=k)
            brb = is_colorable(grph, 11, timer)
            if not brb:
                print(base_conf, add_conf + [elem])
    if how_much == BIG_NUM:
        print("analysing {} finished".format(BIG_NUM))


def bruteforce_nolimits_nok4_second_short(to_check, base_conf, add_conf, k_2, k_4, k2s = [], kl3s=[], kl4s=[], how_much=4, timer=120, n=10, k=6, fromm=0):
    to_check = sort_to_check(to_check, base_conf, add_conf)
    if how_much == BIG_NUM:
        to_check = to_check[fromm:]
    ln = len(to_check)
    for i in range(ln):
        BPM[how_much][0] = i
        BPM[how_much][1] = ln
        elem = to_check[i]
        if elem in base_conf or elem in add_conf:
            continue
        flag = True
        for el in add_conf:
            if dist(elem, el) > 6:
                flag = False
                break
        if not flag:
            continue
        big_flag = True
        for kl3 in kl3s:
            res = 0
            for v in kl3:
                res ^= v
            if res == elem:
                big_flag = False
                break
        if not big_flag:
            continue
        for kl4 in kl4s:
            fflag = False
            for v in kl4:
                if dist(elem, v) < 6:
                    fflag = True
                    break
            if not fflag:
                big_flag = False
                break
        if not big_flag:
            continue
    
        kl4s_updated = []
        for k3ss in kl3s:
            if dist(elem, k3ss[0]) == 6 and dist(elem, k3ss[1]) == 6 and dist(elem, k3ss[2]):
                kl4s_updated.append((k3ss[2], k3ss[0], k3ss[1], elem))
        kl3s_updated = []
        for k2ss in k2s:
            if dist(elem, k2ss[0]) == 6 and dist(elem, k2ss[1]) == 6:
                kl3s_updated.append((k2ss[0], k2ss[1], elem))
        k2s_updated = []
        for v in base_conf + add_conf:
            if dist(elem, v) == 6:
                k2s_updated.append((elem, v))
        grph = simple_cook_graph_no_K4_second(base_conf=base_conf, add_conf=add_conf + [elem], kl3s = kl3s + kl3s_updated, kl4s = kl4s + kl4s_updated, k_2=k_2, k_4=k_4, k_6=[], n=n, k=k)
        brb = is_colorable(grph, 11, timer)
        if not brb:
            if how_much == 1:
                print(base_conf, add_conf + [elem])
        if how_much > 1:
            bruteforce_nolimits_nok4_second_short(to_check[i+1:], base_conf, add_conf + [elem], k_2=k_2,
                                     k_4=k_4, k2s = k2s + k2s_updated, kl3s = kl3s + kl3s_updated, kl4s = kl4s + kl4s_updated,
                                     how_much=how_much-1,timer=timer, n=n, k=k)

    if how_much == BIG_NUM:
        print("analysing {}-or-less-but-at-least-1 finished".format(BIG_NUM))

In [ ]:
%%time
print("Before everything", COLORSS)
k_6 = sort_to_check(k_6, base_conf, [])
print('to check:', len(k_6[:60]), len(k_6))
# bruteforce_nolimits_nok4_second(k_6[:60], base_conf, [], k_2, k_4 + k_6[60:], k2s, kl3s, kl4s, 3, 20, 10, 6)
bruteforce_nolimits_nok4_second_2(k_6, base_conf, [], k_2, k_4, k2s, kl3s, kl4s, 3, 20, 10, 6)
print("After add 3", COLORSS)

In [ ]:
%%time
print("Before add 3, but last after first 75", COLORSS)
bruteforce_nolimits_nok4_second_short(k_6[75:], base_conf, [], k_2, k_4, k2s, kl3s, kl4s, 3, 20, 10, 6)
print("After add 3, but last after first 75", COLORSS)
bruteforce_nolimits_nok4_second_short(k_6, base_conf, [], k_2, k_4, k2s, kl3s, kl4s, 2, 20, 10, 6)
print("After add 2", COLORSS)
bruteforce_nolimits_nok4_second_short(k_6, base_conf, [], k_2, k_4, k2s, kl3s, kl4s, 1, 20, 10, 6)
print("After add 1", COLORSS)

In [ ]:
%%time
grph = simple_cook_graph_no_K4_second(base_conf, [], kl3s, kl4s, k_2, k_4, [], n=10, k=6)
brb = is_colorable(grph, 11, 20)
brb

# SOLVE 4-CLIQUE-1, but no 5-CLIQUE - e on base of 4-CLIQUE-1

In [8]:
allowed = [
    [0, 0, 0, 0],
    [0, 1, 1, 0],
    [1, 0, 1, 0],
    [1, 1, 0, 0],
    [1, 1, 2, 0],
    [1, 2, 1, 0],
    [2, 1, 1, 0],
    [2, 2, 0, 0],
    [2, 2, 2, 0],
]
vertexes = []
for i in range(0, 1024):
    a = dist(i & 0b1110000000, 0)
    b = dist(i & 0b0001110000, 0)
    c = dist(i & 0b0000001110, 0)
    d = dist(i & 0b0000000001, 0)
    flag = False
    for config in allowed:
        if a == config[0] and b == config[1] and c == config[2] and d == config[3]:
            flag = True
    if flag:
        vertexes.append(i)
g = nx.Graph()


for vertex in vertexes:
    g.add_node(vertex)
for i in list(g.nodes()):
    for j in g.nodes():
        if (i<j) and (dist(i,j)==6):
            g.add_edge(i,j)

print(is_colorable(nx.convert_node_labels_to_integers(g), 8, 20))

True


# SOLVE 5-CLIQUE - e on base of 4-CLIQUE-1

## subtask. Add exactly 0, 1 or 2 vertexes from  k6[:60] 

In [ ]:
FILENAME = "NOK5_ISK5WOEDGE_0_1_2"
kv = [0, 504, 63, 455, 658]
tt = []
for i in range(1024):
    if dist(i, 0) % 2 == 1 or dist(i, 0) > 6 or i in kv:
        continue
    chk = 0
    for v in kv:
        if dist(v, i) > 6:
            chk = 10000
        elif dist(v, i) == 6:
            chk += 1
    if chk == 3:
        lhs = 0 ^ 945 ^ 910 ^ 630 ^ i
        flag = True
        for v in kv:
            if lhs ^ v == 0:
                flag = False
        if flag:
            tt.append(i)

In [ ]:
base_conf = [0, 504, 63, 455, 658]

k2s = [(0, 504),
       (0, 63),
       (0, 455),
       (658, 504),
       (658, 63),
       (658, 455),
       (63, 504),
       (455, 504),
       (63, 455)
       ]
kl3s = [(0, 504, 63),
        (0, 504, 455),
        (0, 63, 455),
        (658, 504, 63),
        (658, 504, 455),
        (658, 63, 455),
        (504, 63, 455)
        ]
kl4s = [(0, 504, 63, 455), (504, 63, 455, 658)]

k_2 = []
k_4 = []
k_6 = []

for i in range(2**10):
    if dist(0, i) % 2 == 1:
        continue
    flag = True
    for base_v in base_conf:
        if dist(i, base_v) == 0 or dist(i, base_v) > 6:
            flag = False
            continue
    big_flag = True
    for kl4 in kl4s:
        fflag = False
        for v in kl4:
            if dist(i, v) < 6:
                fflag = True
                break
        if not fflag:
            big_flag = False
            break
    if not big_flag:
        continue
    if flag:
        if dist(0, i) == 2:
            k_2.append(i)
        if dist(0, i) == 4:
            k_4.append(i)
        if dist(0, i) == 6:
            k_6.append(i)

In [ ]:
SUPER_SAVED = []

In [ ]:
BIG_NUM = 3
COUNTER = 0
BPM = [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]

# не симпл
def simple_cook_graph_no_K5(base_conf, add_conf, kl4s, k_2, k_4, k_6, n=10, k=6):   # третий параметр - множество вершин
    global COUNTER, BPM
    COUNTER += 1
    if COUNTER % 50 == 0:
        print(BPM)
    g = nx.Graph() 
    bans = base_conf + add_conf
    pooles = k_2 + k_4 + k_6
    for i in bans:
        g.add_node(i)
    
    good_vrtx = []
    for i in pooles:
        flag = True
        for j in bans:
            if dist(i,j) > k or dist(i, j) == 0:
                flag = False
                break
        if not flag:
            continue

        big_flag = True
        for kl4 in kl4s:
            fflag = False
            for v in kl4:
                if dist(i, v) < 6:
                    fflag = True
                    break
            if not fflag:
                big_flag = False
                break
        if big_flag:
            g.add_node(i)
            good_vrtx.append(i)

    for i in bans:
        for j in bans:
            if (i<j) and (dist(i,j)==k):
                g.add_edge(i,j)
    
    for i in bans:
        for j in good_vrtx:
            if (i<j) and (dist(i,j)==k):
                g.add_edge(i,j)

    for i in good_vrtx:
        for j in bans:
            if (i<j) and (dist(i,j)==k):
                g.add_edge(i,j)

                
    prepared = []
    for i in bans:
        for j in bans:
            for qq in bans:
                if dist(i,j)==k and dist(qq,j)==k and dist(i,qq)==k and i < j and j < qq:
                    prepared.append((i, j, qq))
                
    for i in good_vrtx:
        for j in good_vrtx:
            if (i<j) and (dist(i,j)==k):
                flag = True
                for prep in prepared:
                    if dist(i, prep[0]) == k and dist(i, prep[1]) == k and dist(j, prep[0]) == k and dist(j, prep[1]) == k and dist(i, prep[2]) == k and dist(j, prep[2]) == k:
                        flag = False
                if flag:
                    g.add_edge(i,j)
    
    return nx.convert_node_labels_to_integers(g)

def sort_to_check(to_check, base_conf, add_conf):
    to_to = [[a, 0] for a in to_check]
    for a in to_to:
        for b in base_conf:
            if dist(a[0], b) == 6:
                a[1] += 1
        for b in  add_conf:
            if dist(a[0], b) == 6:
                a[1] += 1
    tbd = sorted(to_to, key=lambda x: (-x[1], x[0]))
    return [a[0] for a in tbd]


def bruteforce_nolimits_nok5(to_check, base_conf, add_conf, k_2, k_4, k2s = [], kl3s=[], kl4s=[], how_much=BIG_NUM, timer=120, n=10, k=6, fromm=0):
    global COUNTER, BPM, SUPER_SAVED
    to_check = sort_to_check(to_check, base_conf, add_conf)
    if how_much == BIG_NUM:
        to_check = to_check[fromm:]
    ln = len(to_check)
    for i in range(ln):
        BPM[how_much][0] = i
        BPM[how_much][1] = ln
        # if i % 20 == 0 and how_much == BIG_NUM - 1:
        #     print(i, len(to_check))
        elem = to_check[i]
        if elem in base_conf or elem in add_conf:
            continue
        flag = True
        for el in add_conf:
            if dist(elem, el) > 6:
                flag = False
                break
        if not flag:
            continue
        big_flag = True
        for kl4 in kl4s:
            fflag = False
            for v in kl4:
                if dist(elem, v) < 6:
                    fflag = True
                    break
            if not fflag:
                big_flag = False
                break
        if not big_flag:
            continue
    
        kl4s_updated = []
        for k3ss in kl3s:
            if dist(elem, k3ss[0]) == 6 and dist(elem, k3ss[1]) == 6 and dist(elem, k3ss[2]):
                kl4s_updated.append((k3ss[2], k3ss[0], k3ss[1], elem))
        kl3s_updated = []
        for k2ss in k2s:
            if dist(elem, k2ss[0]) == 6 and dist(elem, k2ss[1]) == 6:
                kl3s_updated.append((k2ss[0], k2ss[1], elem))
        k2s_updated = []
        for v in base_conf + add_conf:
            if dist(elem, v) == 6:
                k2s_updated.append((elem, v))
        if how_much > 1:
            bruteforce_nolimits_nok5(to_check[i+1:], base_conf, add_conf + [elem], k_2=k_2,
                                     k_4=k_4, k2s = k2s + k2s_updated, kl3s = kl3s + kl3s_updated, kl4s = kl4s + kl4s_updated,
                                     how_much=how_much-1,timer=timer, n=n, k=k)
        else:
            grph = simple_cook_graph_no_K5(base_conf=base_conf, add_conf=add_conf + [elem], kl4s = kl4s + kl4s_updated, k_2=k_2, k_4=k_4, k_6=to_check[i+1:], n=n, k=k)
            brb = is_colorable(grph, 11, timer)
            if not brb:
                print(base_conf, add_conf + [elem])
                SUPER_SAVED.append(add_conf + [elem])
    if how_much == BIG_NUM:
        print("analysing {} finished".format(BIG_NUM))

def bruteforce_nolimits_nok5_short(to_check, base_conf, add_conf, k_2, k_4, k2s = [], kl3s=[], kl4s=[], how_much=4, timer=120, n=10, k=6, fromm=0):
    global COUNTER, BPM, SUPER_SAVED
    to_check = sort_to_check(to_check, base_conf, add_conf)
    if how_much == BIG_NUM:
        to_check = to_check[fromm:]
    ln = len(to_check)
    for i in range(ln):
        BPM[how_much][0] = i
        BPM[how_much][1] = ln
        elem = to_check[i]
        if elem in base_conf or elem in add_conf:
            continue
        flag = True
        for el in add_conf:
            if dist(elem, el) > 6:
                flag = False
                break
        if not flag:
            continue
        big_flag = True
        for kl4 in kl4s:
            fflag = False
            for v in kl4:
                if dist(elem, v) < 6:
                    fflag = True
                    break
            if not fflag:
                big_flag = False
                break
        if not big_flag:
            continue
    
        kl4s_updated = []
        for k3ss in kl3s:
            if dist(elem, k3ss[0]) == 6 and dist(elem, k3ss[1]) == 6 and dist(elem, k3ss[2]):
                kl4s_updated.append((k3ss[2], k3ss[0], k3ss[1], elem))
        kl3s_updated = []
        for k2ss in k2s:
            if dist(elem, k2ss[0]) == 6 and dist(elem, k2ss[1]) == 6:
                kl3s_updated.append((k2ss[0], k2ss[1], elem))
        k2s_updated = []
        for v in base_conf + add_conf:
            if dist(elem, v) == 6:
                k2s_updated.append((elem, v))
        grph = simple_cook_graph_no_K5(base_conf=base_conf, add_conf=add_conf + [elem], kl4s = kl4s + kl4s_updated, k_2=k_2, k_4=k_4, k_6=[], n=n, k=k)
        brb = is_colorable(grph, 11, timer)
        if not brb:
            if how_much == 1:
                print(base_conf, add_conf + [elem])
                SUPER_SAVED.append(add_conf + [elem])
        if how_much > 1:
            bruteforce_nolimits_nok5_short(to_check[i+1:], base_conf, add_conf + [elem], k_2=k_2,
                                     k_4=k_4, k2s = k2s + k2s_updated, kl3s = kl3s + kl3s_updated, kl4s = kl4s + kl4s_updated,
                                     how_much=how_much-1,timer=timer, n=n, k=k)

    if how_much == BIG_NUM:
        print("analysing {}-or-less-but-at-least-1 finished".format(BIG_NUM))

In [ ]:
%%time
BIG_NUM = 2
print(COLORSS)
SUPER_SAVED = []
k_6 = sort_to_check(k_6, base_conf, [])
print('to check:', len(k_6[:60]), len(k_6))
bruteforce_nolimits_nok5_short(k_6[:60], base_conf, [], k_2, k_4 + k_6[60:], k2s, kl3s, kl4s, 2, 20, 10, 6)

super saved was empty. If it will not be empty, probably Fix_Issues blocks will help

In [ ]:
%%time
BIG_NUM = 1
print(COLORSS)
k_6 = sort_to_check(k_6, base_conf, [])
print('to check:', len(k_6[:60]), len(k_6))
bruteforce_nolimits_nok5_short(k_6[:60], base_conf, [], k_2, k_4 + k_6[60:], k2s, kl3s, kl4s, 1, 20, 10, 6)

Fix_Issues 

In [ ]:
def build_blocks_by_conf(conf):
    vrtx = [0, 504, 63, 455, 658] + conf
    lss = len(vrtx)
    k22 = []
    k33 = []
    k44=[]
    for i in range(lss):
        for j in range(i+1, lss):
            if dist(vrtx[i], vrtx[j]) == 6:
                k22.append((vrtx[i], vrtx[j]))
    for i in range(lss):
        for j in range(i+1, lss):
            for j1 in range(j+1, lss):
                if dist(vrtx[i], vrtx[j]) == 6 and dist(vrtx[i], vrtx[j1]) == 6 and dist(vrtx[j1], vrtx[j]) == 6:
                    k33.append((vrtx[i], vrtx[j], vrtx[j1]))
    
    for i in range(lss):
        for j in range(i+1, lss):
            for j1 in range(j+1, lss):
                for j2 in range(j1+1, lss):
                    if dist(vrtx[i], vrtx[j]) == 6 and dist(vrtx[i], vrtx[j1]) == 6 and dist(vrtx[j1], vrtx[j]) == 6 and\
                        dist(vrtx[i], vrtx[j2]) == 6 and dist(vrtx[j2], vrtx[j1]) == 6 and dist(vrtx[j2], vrtx[j]) == 6:
                        k44.append((vrtx[i], vrtx[j], vrtx[j1], vrtx[j2]))
    return k22, k33, k44

In [ ]:
%%time
SSAVED = []
for gr in SUPER_SAVED:
    k2_ss, k3_ss, k4_ss = build_blocks_by_conf(gr)
    BIG_NUM = 1 # why add exactly +1? Because it will be +2 exactly from all k6. +3+ from k6 is covered below.
    bruteforce_nolimits_nok5_short(to_check=k_6[60:], base_conf=[0, 504, 63, 455, 658], add_conf=gr, k_2=k_2, k_4=k_4, k2s=k2_ss, kl3s=k3_ss, kl4s=k4_ss, how_much=BIG_NUM, timer=20, n=10, k=6)


## subtask. Everything else

In [ ]:
%%time
BIG_NUM = 3
print(COLORSS)
SUPER_SAVED = []
k_6 = sort_to_check(k_6, base_conf, [])
bruteforce_nolimits_nok5(k_6, base_conf, [], k_2, k_4, k2s, kl3s, kl4s, 3, 20, 10, 6)

# SOLVE 5-CLIQUE

In [ ]:
BIG_NUM = 3
COUNTER = 0
BPM = [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]

def simple_cook_graph_no_K6(base_conf, add_conf, kl5s, k_2, k_4, k_6, n=10, k=6):   # третий параметр - множество вершин
    global COUNTER, BPM
    COUNTER += 1
    if COUNTER % 50 == 0:
        print(BPM)
    g = nx.Graph() 
    bans = base_conf + add_conf
    pooles = k_2 + k_4 + k_6
    for i in bans:
        g.add_node(i)
    
    for i in pooles:
        flag = True
        for j in bans:
            if dist(i,j) > k or dist(i, j) == 0:
                flag = False
                break
        if not flag:
            continue

        big_flag = True
        for kl5 in kl5s:
            fflag = False
            for v in kl5:
                if dist(i, v) < 6:
                    fflag = True
                    break
            if not fflag:
                big_flag = False
                break
        if big_flag:
            g.add_node(i)
    for i in list(g.nodes()):
        for j in g.nodes():
            if (i<j) and (dist(i,j)==k):
                g.add_edge(i,j)
    return nx.convert_node_labels_to_integers(g)

def sort_to_check(to_check, base_conf, add_conf):
    to_to = [[a, 0] for a in to_check]
    for a in to_to:
        for b in base_conf:
            if dist(a[0], b) == 6:
                a[1] += 1
        for b in  add_conf:
            if dist(a[0], b) == 6:
                a[1] += 1
    tbd = sorted(to_to, key=lambda x: (-x[1], x[0]))
    return [a[0] for a in tbd]


def bruteforce_nolimits_nok6(to_check, base_conf, add_conf, k_2, k_4, k2s = [], kl3s=[], kl4s=[], kl5s=[], how_much=BIG_NUM, timer=120, n=10, k=6, fromm=0):
    global COUNTER, BPM
    to_check = sort_to_check(to_check, base_conf, add_conf)
    if how_much == BIG_NUM:
        to_check = to_check[fromm:]
    ln = len(to_check)
    for i in range(ln):
        BPM[how_much][0] = i
        BPM[how_much][1] = ln
        # if i % 20 == 0 and how_much == BIG_NUM - 1:
        #     print(i, len(to_check))
        elem = to_check[i]
        if elem in base_conf or elem in add_conf:
            continue
        flag = True
        for el in add_conf:
            if dist(elem, el) > 6:
                flag = False
                break
        if not flag:
            continue
        big_flag = True
        for kl5 in kl5s:
            fflag = False
            for v in kl5:
                if dist(elem, v) < 6:
                    fflag = True
                    break
            if not fflag:
                big_flag = False
                break
        if not big_flag:
            continue
    
        kl5s_updated = []
        for k4ss in kl4s:
            if dist(elem, k4ss[0]) == 6 and dist(elem, k4ss[1]) == 6 and dist(elem, k4ss[2]) == 6 and dist(elem, k4ss[3]) == 6:
                kl5s_updated.append((k4ss[3], k4ss[2], k4ss[0], k4ss[1], elem))
        kl4s_updated = []
        for k3ss in kl3s:
            if dist(elem, k3ss[0]) == 6 and dist(elem, k3ss[1]) == 6 and dist(elem, k3ss[2]) == 6:
                kl4s_updated.append((k3ss[2], k3ss[0], k3ss[1], elem))
        kl3s_updated = []
        for k2ss in k2s:
            if dist(elem, k2ss[0]) == 6 and dist(elem, k2ss[1]) == 6:
                kl3s_updated.append((k2ss[0], k2ss[1], elem))
        k2s_updated = []
        for v in base_conf + add_conf:
            if dist(elem, v) == 6:
                k2s_updated.append((elem, v))
        if how_much > 1:
            bruteforce_nolimits_nok6(to_check[i+1:], base_conf, add_conf + [elem], k_2=k_2,
                                     k_4=k_4, k2s = k2s + k2s_updated, kl3s = kl3s + kl3s_updated, kl4s = kl4s + kl4s_updated, kl5s = kl5s + kl5s_updated,
                                     how_much=how_much-1,timer=timer, n=n, k=k)
        else:
            grph = simple_cook_graph_no_K6(base_conf=base_conf, add_conf=add_conf + [elem], kl5s = kl5s + kl5s_updated, k_2=k_2, k_4=k_4, k_6=to_check[i+1:], n=n, k=k)
            brb = is_colorable(grph, 11, timer)
            if not brb:
                print(base_conf, add_conf + [elem])
    if how_much == BIG_NUM:
        print("analysing {} finished".format(BIG_NUM))

def bruteforce_nolimits_nok6_short(to_check, base_conf, add_conf, k_2, k_4, k2s = [], kl3s=[], kl4s=[], kl5s=[], how_much=4, timer=120, n=10, k=6, fromm=0):
    global COUNTER, BPM
    to_check = sort_to_check(to_check, base_conf, add_conf)
    if how_much == BIG_NUM:
        to_check = to_check[fromm:]
    ln = len(to_check)
    for i in range(ln):
        BPM[how_much][0] = i
        BPM[how_much][1] = ln
        elem = to_check[i]
        if elem in base_conf or elem in add_conf:
            continue
        flag = True
        for el in add_conf:
            if dist(elem, el) > 6:
                flag = False
                break
        if not flag:
            continue
        big_flag = True
        for kl4 in kl5s:
            fflag = False
            for v in kl4:
                if dist(elem, v) < 6:
                    fflag = True
                    break
            if not fflag:
                big_flag = False
                break
        if not big_flag:
            continue

        kl5s_updated = []
        for k4ss in kl4s:
            if dist(elem, k4ss[0]) == 6 and dist(elem, k4ss[1]) == 6 and dist(elem, k4ss[2]) == 6 and dist(elem, k4ss[3]) == 6:
                kl5s_updated.append((k4ss[3], k4ss[2], k4ss[0], k4ss[1], elem))
        kl4s_updated = []
        for k3ss in kl3s:
            if dist(elem, k3ss[0]) == 6 and dist(elem, k3ss[1]) == 6 and dist(elem, k3ss[2]) == 6:
                kl4s_updated.append((k3ss[2], k3ss[0], k3ss[1], elem))
        kl3s_updated = []
        for k2ss in k2s:
            if dist(elem, k2ss[0]) == 6 and dist(elem, k2ss[1]) == 6:
                kl3s_updated.append((k2ss[0], k2ss[1], elem))
        k2s_updated = []
        for v in base_conf + add_conf:
            if dist(elem, v) == 6:
                k2s_updated.append((elem, v))
        grph = simple_cook_graph_no_K6(base_conf=base_conf, add_conf=add_conf + [elem], kl5s = kl5s + kl5s_updated, k_2=k_2, k_4=k_4, k_6=[], n=n, k=k)
        brb = is_colorable(grph, 11, timer)
        if not brb:
            if how_much == 1:
                print(base_conf, add_conf + [elem])
        if how_much > 1:
            bruteforce_nolimits_nok6_short(to_check[i+1:], base_conf, add_conf + [elem], k_2=k_2,
                                     k_4=k_4, k2s = k2s + k2s_updated, kl3s = kl3s + kl3s_updated, kl4s = kl4s + kl4s_updated, kl5s = kl5s + kl5s_updated,
                                     how_much=how_much-1,timer=timer, n=n, k=k)

    if how_much == BIG_NUM:
        print("analysing {}-or-less-but-at-least-1 finished".format(BIG_NUM))

In [ ]:
base_conf = [0, 945, 910, 630, 219]

k2s = [(0, 945),
       (0, 910),
       (0, 630),
       (0, 219),
       (219, 945),
       (219, 910),
       (219, 630),
       (630, 945),
       (630, 910),
       (910, 945)
       ]
kl3s = [(0, 0, 945),
        (0, 0, 910),
        (0, 0, 630),
        (0, 0, 219),
        (0, 945, 910),
        (0, 945, 630),
        (0, 945, 219),
        (0, 910, 630),
        (0, 910, 219),
        (0, 630, 219),
        (945, 945, 910),
        (945, 945, 630),
        (945, 945, 219),
        (945, 910, 630),
        (945, 910, 219),
        (945, 630, 219),
        (910, 910, 630),
        (910, 910, 219),
        (910, 630, 219),
        (630, 630, 219)]
kl4s = [(945, 910, 630, 219), (0, 910, 630, 219), (0, 945, 630, 219), (0, 945, 910, 219), (0, 945, 910, 630)]
kl5s = [(0, 945, 910, 630, 219)]

k_2 = []
k_4 = []
k_6 = []

for i in range(2**10):
    if dist(0, i) % 2 == 1:
        continue
    flag = True
    for base_v in base_conf:
        if dist(i, base_v) == 0 or dist(i, base_v) > 6:
            flag = False
            continue
    big_flag = True
    for kl4 in kl4s:
        fflag = False
        for v in kl4:
            if dist(i, v) < 6:
                fflag = True
                break
        if not fflag:
            big_flag = False
            break
    if not big_flag:
        continue
    if flag:
        if dist(0, i) == 2:
            k_2.append(i)
        if dist(0, i) == 4:
            k_4.append(i)
        if dist(0, i) == 6:
            k_6.append(i)

In [ ]:
%%time
print("COLORED", COLORSS)
k_6 = sort_to_check(k_6, base_conf, [])
print('to check:', len(k_6[:60]), len(k_6))
bruteforce_nolimits_nok6(k_6[:60], base_conf, [], k_2, k_4 + k_6[60:], k2s, kl3s, kl4s, kl5s, BIG_NUM, 20, 10, 6)
print("AFTER 3-COLORED", COLORSS)

In [ ]:
%%time
BIG_NUM=3
bruteforce_nolimits_nok6_short(k_6[:60], base_conf, [], k_2, k_4 + k_6[60:], k2s, kl3s, kl4s, kl5s, BIG_NUM - 1, 20, 10, 6)
print("2 finished")
print("AFTER 2-COLORED", COLORSS)
bruteforce_nolimits_nok6_short(k_6[:60], base_conf, [], k_2, k_4 + k_6[60:], k2s, kl3s, kl4s, kl5s, BIG_NUM - 2, 20, 10, 6)
print("1 finished")
print("AFTER 1-COLORED", COLORSS)

In [ ]:
%%time
grph = simple_cook_graph_no_K6(base_conf=base_conf,
                        add_conf=[],
                        kl5s = kl5s,
                        k_2=k_2,
                        k_4=k_4,
                        k_6=[], n=10, k=6)
brb = is_colorable(grph, 11, 20)
brb

# SOLVE 6-CLIQUE

## Solve 6-Clique + 1.

In [ ]:
FILENAME = "ISK6"

BIG_NUM = 3
COUNTER = 0
BPM = [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]

def simple_cook_graph_K6(base_conf, add_conf, k_2, k_4, k_6, n=10, k=6):   # третий параметр - множество вершин
    global COUNTER, BPM
    COUNTER += 1
#     if COUNTER % 500 == 0:
#         print(BPM)
    g = nx.Graph() 
    bans = base_conf + add_conf
    pooles = k_2 + k_4 + k_6
    for i in bans:
        g.add_node(i)
    
    for i in pooles:
        flag = True
        for j in bans:
            if dist(i,j) > k or dist(i, j) == 0:
                flag = False
                break
        if not flag:
            continue
        g.add_node(i)
    for i in list(g.nodes()):
        for j in g.nodes():
            if (i<j) and (dist(i,j)==k):
                g.add_edge(i,j)
    return nx.convert_node_labels_to_integers(g)

def sort_to_check(to_check, base_conf, add_conf):
    to_to = [[a, 0] for a in to_check]
    for a in to_to:
        for b in base_conf:
            if dist(a[0], b) == 6:
                a[1] += 1
        for b in  add_conf:
            if dist(a[0], b) == 6:
                a[1] += 1
    tbd = sorted(to_to, key=lambda x: (-x[1], x[0]))
    return [a[0] for a in tbd]


def bruteforce_nolimits_k6(to_check, base_conf, add_conf, k_2, k_4, how_much=BIG_NUM, timer=120, n=10, k=6, fromm=0):
    global COUNTER, BPM
    to_check = sort_to_check(to_check, base_conf, add_conf)
    if how_much == BIG_NUM:
        to_check = to_check[fromm:]
    ln = len(to_check)
    for i in range(ln):
        BPM[how_much][0] = i
        BPM[how_much][1] = ln
        # if i % 20 == 0 and how_much == BIG_NUM - 1:
        #     print(i, len(to_check))
        elem = to_check[i]
        if elem in base_conf or elem in add_conf:
            continue
        flag = True
        for el in add_conf:
            if dist(elem, el) > 6:
                flag = False
                break
        if not flag:
            continue

        if how_much > 1:
            bruteforce_nolimits_k6(to_check[i+1:], base_conf, add_conf + [elem], k_2=k_2,
                                     k_4=k_4, 
                                     how_much=how_much-1,timer=timer, n=n, k=k)
        else:
            grph = simple_cook_graph_K6(base_conf=base_conf, add_conf=add_conf + [elem], k_2=k_2, k_4=k_4, k_6=to_check[i+1:], n=n, k=k)
            brb = is_colorable(grph, 11, timer)
            if not brb:
                print(base_conf, add_conf + [elem])
    if how_much == BIG_NUM:
        print("analysing {} finished".format(BIG_NUM))

def bruteforce_nolimits_k6_short(to_check, base_conf, add_conf, k_2, k_4, how_much=4, timer=120, n=10, k=6, fromm=0):
    global COUNTER, BPM
    to_check = sort_to_check(to_check, base_conf, add_conf)
    if how_much == BIG_NUM:
        to_check = to_check[fromm:]
    ln = len(to_check)
    for i in range(ln):
        BPM[how_much][0] = i
        BPM[how_much][1] = ln
        elem = to_check[i]
        if elem in base_conf or elem in add_conf:
            continue
        flag = True
        for el in add_conf:
            if dist(elem, el) > 6:
                flag = False
                break
        if not flag:
            continue
    
        grph = simple_cook_graph_K6(base_conf=base_conf, add_conf=add_conf + [elem], k_2=k_2, k_4=k_4, k_6=[], n=n, k=k)
        brb = is_colorable(grph, 11, timer)
        if not brb:
            if how_much == 1:
                print(base_conf, add_conf + [elem])
        if how_much > 1:
            bruteforce_nolimits_k6_short(to_check[i+1:], base_conf, add_conf + [elem], k_2=k_2,
                                     k_4=k_4, how_much=how_much-1,timer=timer, n=n, k=k)

    if how_much == BIG_NUM:
        print("analysing {}-or-less-but-at-least-1 finished".format(BIG_NUM))

In [ ]:
kv = [0, 945, 910, 630, 219, 365, 159]
k_6 = []
k_4 = []
k_2 = []
for i in range(1024):
    if dist(i, 0) % 2 == 1 or dist(i, 0) > 6 or i in kv:
        continue
    flag = True
    for v in kv:
        if dist(v, i) > 6:
            flag = False
    if not flag:
        continue
    if dist(i, 0) == 6:
        k_6.append(i)
    elif dist(i, 0) == 4:
        k_4.append(i)
    elif dist(i, 0) == 2:
        k_2.append(i)
print(len(k_6), len(k_4), len(k_2))

In [ ]:
%%time
BIG_NUM=4
print("COLORED", COLORSS)
print('to check:', len(k_6), len(k_6))
bruteforce_nolimits_k6(k_6, kv, [], k_2, k_4, BIG_NUM, 20, 10, 6)
print("AFTER ALL-COLORED", COLORSS)

In [ ]:
%%time
BIG_NUM=3
print("COLORED", COLORSS)
print('to check:', len(k_6), len(k_6))
bruteforce_nolimits_k6_short(k_6, kv, [], k_2, k_4, BIG_NUM, 20, 10, 6)
print("AFTER ALL-COLORED", COLORSS)

In [ ]:
%%time
BIG_NUM=2
print("COLORED", COLORSS)
print('to check:', len(k_6), len(k_6))
bruteforce_nolimits_k6_short(k_6, kv, [], k_2, k_4, BIG_NUM, 20, 10, 6)
print("AFTER ALL-COLORED", COLORSS)

In [ ]:
%%time
BIG_NUM=1
print("COLORED", COLORSS)
print('to check:', len(k_6), len(k_6))
bruteforce_nolimits_k6_short(k_6, kv, [], k_2, k_4, BIG_NUM, 20, 10, 6)
print("AFTER ALL-COLORED", COLORSS)

In [ ]:
%%time
grph = simple_cook_graph_K6(kv, [], k_2, k_4, [], [])
print(is_colorable(grph, 11, 120))

## Solve 6-Clique, no 246666 exists

In [7]:
kv = [0, 945, 910, 630, 219, 365]
k_6 = []
k_4 = []
k_2 = []
for i in range(1024):
    if dist(i, 0) % 2 == 1 or dist(i, 0) > 6 or i in kv:
        continue
    flag = True
    for v in kv:
        if dist(v, i) > 6:
            flag = False
    if not flag:
        continue
    if dist(i, 0) == 6:
        flag = False
        for v in kv:
            if dist(v, i) == 2:
                flag = True
        if not flag:
            k_6.append(i)
    elif dist(i, 0) == 4:
        flag = False
        for v in kv:
            if dist(v, i) == 2:
                flag = True
        if not flag:
            k_4.append(i)
print(len(k_6), len(k_4), len(k_2))

70 70 0


In [ ]:
%%time
BIG_NUM=4
print("COLORED", COLORSS)
print('to check:', len(k_6), len(k_6))
bruteforce_nolimits_k6(k_6, kv, [], k_2, k_4, BIG_NUM, 20, 10, 6)
print("AFTER ALL-COLORED", COLORSS)

In [ ]:
%%time
BIG_NUM=3
print("COLORED", COLORSS)
print('to check:', len(k_6), len(k_6))
bruteforce_nolimits_k6_short(k_6, kv, [], k_2, k_4, BIG_NUM, 20, 10, 6)
print("AFTER ALL-COLORED", COLORSS)

In [ ]:
%%time
BIG_NUM=2
print("COLORED", COLORSS)
print('to check:', len(k_6), len(k_6))
bruteforce_nolimits_k6_short(k_6, kv, [], k_2, k_4, BIG_NUM, 20, 10, 6)
print("AFTER ALL-COLORED", COLORSS)

In [ ]:
%%time
BIG_NUM=1
print("COLORED", COLORSS)
print('to check:', len(k_6), len(k_6))
bruteforce_nolimits_k6_short(k_6, kv, [], k_2, k_4, BIG_NUM, 20, 10, 6)
print("AFTER ALL-COLORED", COLORSS)

In [ ]:
%%time
grph = simple_cook_graph_K6(kv, [], k_2, k_4, [], [])
print(is_colorable(grph, 11, 120))